# Avaliando o modelo fine tunado com o MMLU

In [1]:
!pip install -q transformers==4.40.1
!pip install -q peft==0.10.0
!pip install -q bitsandbytes==0.46.0
!pip install -q accelerate==0.29.3
!pip install -q datasets==2.19.0
!pip install -q trl==0.8.6
!pip install -q huggingface_hub==0.22.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 70.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 77.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 4.1.0 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.40.1 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 106.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 87.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 61.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s 

In [2]:
from datasets import load_dataset
mmlu_dataset = load_dataset("cais/mmlu", "all")
mmlu_dataset

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating test split:   0%|          | 0/14042 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1531 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/285 [00:00<?, ? examples/s]

Generating auxiliary_train split:   0%|          | 0/99842 [00:00<?, ? examples/s]

DatasetDict({
    test: Dataset({
        features: ['question', 'subject', 'choices', 'answer'],
        num_rows: 14042
    })
    validation: Dataset({
        features: ['question', 'subject', 'choices', 'answer'],
        num_rows: 1531
    })
    dev: Dataset({
        features: ['question', 'subject', 'choices', 'answer'],
        num_rows: 285
    })
    auxiliary_train: Dataset({
        features: ['question', 'subject', 'choices', 'answer'],
        num_rows: 99842
    })
})

Pegando 150 questões do MMLU de teste. Importante dizer que com a seed 21, sempre saíra as mesmas questões

In [3]:
import random

mmlu_test_set = mmlu_dataset["test"]

indices = list(range(len(mmlu_test_set)))

random.seed(21)
random.shuffle(indices)

num_questions_to_select = 150
selected_indices = indices[:num_questions_to_select]
selected_mmlu_questions = mmlu_test_set.select(selected_indices)
selected_mmlu_questions

Dataset({
    features: ['question', 'subject', 'choices', 'answer'],
    num_rows: 150
})

In [4]:
selected_mmlu_questions[0]

{'question': 'This question refers to the following information.\nPerestroika [Restructuring] is an urgent necessity arising from the profound processes of development in our socialist society. This society is ripe for change. It has long been yearning for it. Any delay in beginning perestroika could have led to an exacerbated internal situation in the near future, which, to put it bluntly, would have been fraught with serious social, economic, and political crises.\nMikhail Gorbachev, Perestroika: New Thinking for Our Country and the World, 1987\nFrom the passage, one may infer that Gorbachev believed that',
 'subject': 'high_school_european_history',
 'choices': ['the problems that required perestroika were the fault of capitalist enemies of socialism',
  'the problems that required perestroika were internal to the development of socialist society',
  'a socialist society could not work',
  'a socialist society could not coexist with capitalism'],
 'answer': 1}

**IMPORTANTE:** Troque "hf_YOUR_TOKEN_HERE" pelo seu próprio token do hugging face. Para ter acesso ao mistral, você precisa concordar com os termos no site

https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2

In [5]:
from huggingface_hub import login

# Substitua 'hf_YOUR_TOKEN_HERE' pelo seu token real do Hugging Face
login(token='hf_YOUR_TOKEN_HERE')

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


**IMPORTANTE:** Para prosseguir, você precisa baixar o zip que se encontra no nosso drive. Ele é o adaptador do nosso modelo QLoRA. Procure pelo arquivo `mistral_spider_qlora_4.zip`:

Drive: https://drive.google.com/drive/folders/1CADgJH4JmLiwoa6Tb053JXF71F1S7lkG?usp=sharing

In [6]:
!mkdir mistral_spider_qlora_4
!unzip mistral_spider_qlora_4.zip -d mistral_spider_qlora_4

Archive:  mistral_spider_qlora_4.zip
  inflating: mistral_spider_qlora_4/adapter_config.json  
  inflating: mistral_spider_qlora_4/adapter_model.safetensors  
  inflating: mistral_spider_qlora_4/README.md  
  inflating: mistral_spider_qlora_4/special_tokens_map.json  
  inflating: mistral_spider_qlora_4/tokenizer.json  
  inflating: mistral_spider_qlora_4/tokenizer.model  
  inflating: mistral_spider_qlora_4/tokenizer_config.json  
  inflating: mistral_spider_qlora_4/training_args.bin  


Carregando o modelo

In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig, PeftModel
import torch

model_id = "mistralai/Mistral-7B-Instruct-v0.2"
adapter_dir = "mistral_spider_qlora_4"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto", # Automatically maps model to available devices
    trust_remote_code=True,
)

tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

model = PeftModel.from_pretrained(model, adapter_dir)

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [17]:
from transformers import pipeline

pipeline_generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16 if torch.cuda.is_available() and torch.cuda.get_device_capability()[0] >= 8 else torch.float16,
    device_map="auto",
)


answers = ['A', 'B', 'C', 'D']

# função que gera as respostas pro MMLU
def generate_response(question):
  text = f"""[INST]
You are an AI specialized in Multitask Language Understanding.
Based on a textual question, generate ONLY the answer to the question.
Do not write explanations. ONLY write the response letter.

Example 1
Q: {mmlu_dataset['dev'][0]['question']}
A. {mmlu_dataset['dev'][0]['choices'][0]}
B. {mmlu_dataset['dev'][0]['choices'][1]}
C. {mmlu_dataset['dev'][0]['choices'][2]}
D. {mmlu_dataset['dev'][0]['choices'][3]}
Answer: {answers[mmlu_dataset['dev'][0]['answer']]}

Example 2
Q: {mmlu_dataset['dev'][50]['question']}
A. {mmlu_dataset['dev'][50]['choices'][0]}
B. {mmlu_dataset['dev'][50]['choices'][1]}
C. {mmlu_dataset['dev'][50]['choices'][2]}
D. {mmlu_dataset['dev'][50]['choices'][3]}
Answer: {answers[mmlu_dataset['dev'][50]['answer']]}

Example 3
Q: {mmlu_dataset['dev'][100]['question']}
A. {mmlu_dataset['dev'][100]['choices'][0]}
B. {mmlu_dataset['dev'][100]['choices'][1]}
C. {mmlu_dataset['dev'][100]['choices'][2]}
D. {mmlu_dataset['dev'][100]['choices'][3]}
Answer: {answers[mmlu_dataset['dev'][100]['answer']]}

Example 4
Q: {mmlu_dataset['dev'][150]['question']}
A. {mmlu_dataset['dev'][150]['choices'][0]}
B. {mmlu_dataset['dev'][150]['choices'][1]}
C. {mmlu_dataset['dev'][150]['choices'][2]}
D. {mmlu_dataset['dev'][150]['choices'][3]}
Answer: {answers[mmlu_dataset['dev'][150]['answer']]}

New question
Q: {question['question']}
A. {question['choices'][0]}
B. {question['choices'][1]}
C. {question['choices'][2]}
D. {question['choices'][3]}
Answer:
[/INST]"""


  full_response = pipeline_generator(
    text,
    max_new_tokens=16, # Número máximo de tokens a serem gerados
    do_sample=True,
    temperature=0.1,
    top_k=50,
    top_p=0.95,
    num_return_sequences=1,
  )

  full_response = full_response[0]["generated_text"]

  #encontrando a resposta do modelo
  start_of_response_tag = "[/INST]"
  start_index = full_response.find(start_of_response_tag)

  try:
    # A resposta real começa logo após o [/INST] e um possível espaço.
    # Adicione o comprimento do tag e do espaço para pegar o início da resposta.
    model_raw_answer = full_response[start_index + len(start_of_response_tag):].strip()

    # O Mistral muitas vezes termina a resposta do modelo com </s>.
    # Remova esse token se ele estiver presente no final.
    if model_raw_answer.endswith("</s>"):
        model_answer = model_raw_answer[:-len("</s>")].strip()
    else:
        model_answer = model_raw_answer.strip()

    if "[/INST]" in model_answer:
      start_index = model_answer.find("[/INST]")
      model_answer = model_answer[:start_index]

    model_answer = model_answer[0]

  except:
    model_answer = "Could not parse response."

  return model_answer

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'JambaForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MusicgenMelodyFo

In [16]:
generate_response(selected_mmlu_questions[0])

'B'

Calculando o score do modelo fine tunado (quando rodamos, o score foi 66/150)

In [18]:
score = 0
for i in selected_mmlu_questions:
  response = generate_response(i)
  if response != answers[i['answer']]:
    print(i['question'])
    print(f"Predicted answer: {response}")
    print(f"Real answer: {answers[i['answer']]}\n")
  else:
    score += 1
    print(f"Correct! Score: {score}")

print(f"Final score: {score}/150")

Correct! Score: 1
Correct! Score: 2
Correct! Score: 3
Correct! Score: 4
Which of the following is true about dietary calcium:

Predicted answer: A
Real answer: D

Correct! Score: 5
Which of the following statements explains how a point mutation can have no negative impact in the protein formed from a sequence in DNA?
Predicted answer: S
Real answer: B

Denise Developer found a near-perfect piece of property zoned commercial in the downtown area of Pleasantville. She hired an architect to design a high tech office building with an objective of attracting upscale tech-industry tenants. She approached Microhard and secured a lease agreement for over half of the available square footage. The lease was not recorded, but the agreement allowed Denise to attract other wealthy tenants. She also disclosed the Microhard lease on her application for a $1,500,000 construction loan from Maximum Mortgage Company. Denise executed a mortgage and related promissory note in favor of Maximum. Maximum reco

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Correct! Score: 6
 Construct a complete truth table for the following argument. Then, using the truth table, determine whether the argument is valid or invalid. If the argument is invalid, choose an option which presents a counterexample. (There may be other counterexamples as well.)
J ≡ ~K
J ∨ L
~K / L
Predicted answer: S
Real answer: C

Correct! Score: 7
 Just war theory's principle of military necessity belongs to
Predicted answer: B
Real answer: A

A defendant was arrested and prosecuted for the crime of false pretenses. At trial, the defendant testified that she prophesied the victim's imminent death. The defendant admitted that she told the victim she could save him if he brought a large sum of money wrapped in a handkerchief to a ritual. After the ritual, the defendant returned the handkerchief to the victim filled with waste paper rather than the money. After the defendant was convicted of false pretenses, she moved for a new trial and offered the affidavits of several jurors w